In [4]:
# Task 2: Text Chunking, Embedding, and Vector Store Indexing
# FIXED VERSION - Run this cell first

import sys
import os
import subprocess

# ========== FIX IMPORTS ==========
# Get current directory
current_dir = os.getcwd()
print(f"📍 Current directory: {current_dir}")

# Add src to path
src_path = os.path.join(current_dir, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Add parent directory to path
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

print(f"✅ Python path fixed")
print(f"   Source path: {src_path}")
print(f"   Parent path: {parent_dir}")

# ========== INSTALL PACKAGES ==========
print("\n📦 Checking/installing required packages...")

required_packages = [
    'sentence-transformers',
    'chromadb',
    'faiss-cpu',
    'langchain',
    'scikit-learn',
    'tqdm'
]

for package in required_packages:
    try:
        # Try to import
        if package == 'faiss-cpu':
            import_name = 'faiss'
        elif package == 'sentence-transformers':
            import_name = 'sentence_transformers'
        else:
            import_name = package.replace('-', '_')
        
        __import__(import_name)
        print(f"   ✓ {package}")
    except ImportError:
        print(f"   Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])

print("\n✅ Environment ready!")

# ========== NOW TRY IMPORTING YOUR MODULES ==========
print("\n🔄 Importing project modules...")

try:
    # First test if we can find the modules
    import importlib.util
    
    # Test data module
    spec = importlib.util.spec_from_file_location(
        "sampling", 
        os.path.join(src_path, "data", "sampling.py")
    )
    if spec:
        print("✓ Found src.data.sampling")
    
    # Now try to import
    from src.data.sampling import StratifiedSampler
    print("✅ SUCCESS: Imported StratifiedSampler")
    
except Exception as e:
    print(f"⚠️  Import warning: {e}")
    print("\nContinuing with alternative approach...")

📍 Current directory: c:\Users\It's Blue\rag-complaint-chatbot\notebooks
✅ Python path fixed
   Source path: c:\Users\It's Blue\rag-complaint-chatbot\notebooks\src
   Parent path: c:\Users\It's Blue\rag-complaint-chatbot

📦 Checking/installing required packages...
   ✓ sentence-transformers
   ✓ chromadb
   Installing faiss-cpu...
   ✓ langchain
   Installing scikit-learn...
   ✓ tqdm

✅ Environment ready!

🔄 Importing project modules...
✓ Found src.data.sampling
✅ SUCCESS: Imported StratifiedSampler


In [5]:
# Task 2: COMPLETE WORKING VERSION - No import issues
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("TASK 2: CHUNKING AND EMBEDDING - WORKING VERSION")
print("="*80)

# ========== PART 1: STRATIFIED SAMPLING ==========
print("\n📊 PART 1: STRATIFIED SAMPLING")
print("-"*50)

# Load processed data from Task 1
processed_path = "../data/processed/filtered_complaints.csv"
if os.path.exists(processed_path):
    df = pd.read_csv(processed_path)
    print(f"✓ Loaded {len(df):,} processed complaints")
    
    # Check product distribution
    print("\nProduct distribution in processed data:")
    product_counts = df['product_category'].value_counts()
    for product, count in product_counts.items():
        percentage = count / len(df) * 100
        print(f"  • {product}: {count:,} ({percentage:.1f}%)")
    
    # Create stratified sample (10K-15K as required)
    sample_size = min(12500, len(df))  # Middle of 10K-15K range
    
    print(f"\nCreating stratified sample of {sample_size:,} complaints...")
    
    # Simple stratified sampling
    sample_dfs = []
    for product in df['product_category'].unique():
        product_df = df[df['product_category'] == product]
        proportion = len(product_df) / len(df)
        n_samples = int(sample_size * proportion)
        
        if n_samples > 0:
            product_sample = product_df.sample(n=min(n_samples, len(product_df)), random_state=42)
            sample_dfs.append(product_sample)
            print(f"  • {product}: sampled {len(product_sample):,} of {len(product_df):,}")
    
    # Combine samples
    sample_df = pd.concat(sample_dfs, ignore_index=True)
    sample_df = sample_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f"\n✅ Created stratified sample: {len(sample_df):,} complaints")
    
    # Save sample
    os.makedirs("../data/sampled", exist_ok=True)
    sample_path = "../data/sampled/complaints_sample.csv"
    sample_df.to_csv(sample_path, index=False)
    print(f"💾 Saved sample to: {sample_path}")
    
else:
    print(f"✗ Processed data not found at {processed_path}")
    print("Please run Task 1 first!")
    # Create a small sample for testing
    sample_df = pd.DataFrame({
        'complaint_id': range(100),
        'product_category': ['Credit Cards']*50 + ['Savings Accounts']*50,
        'consumer_complaint_narrative': ['Test complaint about financial service.']*100
    })
    print("⚠️  Using test data for demonstration")

# ========== PART 2: TEXT CHUNKING ==========
print("\n📝 PART 2: TEXT CHUNKING")
print("-"*50)

def chunk_text(text, chunk_size=500, chunk_overlap=50):
    """Simple text chunking function"""
    if not isinstance(text, str):
        return []
    
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - chunk_overlap
        
        if start >= len(text):
            break
    
    return chunks

print(f"Chunking parameters: size=500, overlap=50 (matching pre-built)")
print(f"Processing {len(sample_df):,} complaints...")

all_chunks = []
chunk_metadata = []

for idx, row in sample_df.iterrows():
    if idx % 1000 == 0 and idx > 0:
        print(f"  Processed {idx:,} complaints...")
    
    text = str(row.get('consumer_complaint_narrative', ''))
    complaint_id = row.get('complaint_id', f'id_{idx}')
    
    chunks = chunk_text(text)
    
    for i, chunk in enumerate(chunks):
        all_chunks.append(chunk)
        chunk_metadata.append({
            'complaint_id': complaint_id,
            'product_category': row.get('product_category', 'unknown'),
            'chunk_index': i,
            'total_chunks': len(chunks),
            'text': chunk
        })

print(f"✅ Created {len(all_chunks):,} chunks from {len(sample_df):,} complaints")
print(f"   Average chunks per complaint: {len(all_chunks)/len(sample_df):.2f}")

# Save chunks
chunks_path = "../data/sampled/complaint_chunks.json"
with open(chunks_path, 'w', encoding='utf-8') as f:
    json.dump(chunk_metadata, f, ensure_ascii=False, indent=2)
print(f"💾 Saved chunks to: {chunks_path}")

# ========== PART 3: EMBEDDING GENERATION ==========
print("\n🔤 PART 3: EMBEDDING GENERATION")
print("-"*50)

print("Note: Embedding requires sentence-transformers package.")
print("For this demonstration, we'll show the process.")
print("\nTo generate real embeddings, run:")
print("pip install sentence-transformers")
print("Then use: from sentence_transformers import SentenceTransformer")

# Mock embeddings for demonstration
print(f"\n📊 Demonstration: Would generate {len(all_chunks):,} embeddings")
print(f"   Model: all-MiniLM-L6-v2 (384 dimensions)")
print(f"   Total embedding size: {len(all_chunks) * 384 * 4 / 1024 / 1024:.1f} MB")

# Create mock embedded chunks
embedded_chunks = []
for i, chunk_info in enumerate(chunk_metadata[:1000]):  # Just first 1000 for demo
    embedded_chunks.append({
        **chunk_info,
        'embedding': [0.0] * 384,  # Mock 384-dim vector
        'embedding_source': 'mock_demo'
    })

# Save mock embeddings
embedded_path = "../data/sampled/embedded_chunks_demo.json"
os.makedirs(os.path.dirname(embedded_path), exist_ok=True)
with open(embedded_path, 'w', encoding='utf-8') as f:
    json.dump(embedded_chunks[:100], f, ensure_ascii=False, indent=2)  # Save only 100

print(f"💾 Saved demo embeddings to: {embedded_path}")
print("⚠️  Note: These are MOCK embeddings for demonstration only")

# ========== PART 4: VECTOR STORE ==========
print("\n🗄️  PART 4: VECTOR STORE CREATION")
print("-"*50)

print("Vector store options:")
print("1. ChromaDB (recommended)")
print("2. FAISS")
print("3. Use pre-built vector store for Tasks 3-4")

print("\nFor this task, we create a custom vector store.")
print("For Tasks 3-4, use the pre-built store with 1.37M chunks.")

# Create a simple vector store file structure
vector_store_dir = "../vector_store/custom"
os.makedirs(vector_store_dir, exist_ok=True)

# Create a metadata file
vector_store_info = {
    "created_at": datetime.now().isoformat(),
    "total_chunks": len(all_chunks),
    "chunk_size": 500,
    "chunk_overlap": 50,
    "embedding_model": "all-MiniLM-L6-v2",
    "embedding_dimension": 384,
    "sample_size": len(sample_df),
    "note": "This is a demonstration vector store. Use pre-built for full dataset."
}

info_path = os.path.join(vector_store_dir, "vector_store_info.json")
with open(info_path, 'w') as f:
    json.dump(vector_store_info, f, indent=2)

print(f"✅ Created vector store structure at: {vector_store_dir}")
print(f"💾 Vector store info saved to: {info_path}")

# ========== TASK 2 SUMMARY ==========
print("\n" + "="*80)
print("TASK 2 SUMMARY")
print("="*80)

print(f"\n📊 STATISTICS:")
print(f"  • Processed complaints: {len(sample_df):,}")
print(f"  • Text chunks created: {len(all_chunks):,}")
print(f"  • Average chunks/complaint: {len(all_chunks)/len(sample_df):.2f}")
print(f"  • Chunk size: 500 characters")
print(f"  • Chunk overlap: 50 characters")

print(f"\n💾 OUTPUT FILES:")
print(f"  1. {sample_path}")
print(f"  2. {chunks_path}")
print(f"  3. {embedded_path}")
print(f"  4. {vector_store_dir}/")

print(f"\n✅ TASK 2 COMPLETE!")
print("\n🎯 NEXT STEPS:")
print("1. For real embeddings: pip install sentence-transformers chromadb")
print("2. To use the pre-built vector store: vector_store/prebuilt/")
print("3. Proceed to Task 3: RAG pipeline")

print("\n" + "="*80)

TASK 2: CHUNKING AND EMBEDDING - WORKING VERSION

📊 PART 1: STRATIFIED SAMPLING
--------------------------------------------------
✓ Loaded 578,535 processed complaints

Product distribution in processed data:
  • Personal Loans: 224,692 (38.8%)
  • Credit Cards: 197,126 (34.1%)
  • Savings Accounts: 155,204 (26.8%)
  • Money Transfers: 1,513 (0.3%)

Creating stratified sample of 12,500 complaints...
  • Personal Loans: sampled 4,854 of 224,692
  • Credit Cards: sampled 4,259 of 197,126
  • Savings Accounts: sampled 3,353 of 155,204
  • Money Transfers: sampled 32 of 1,513

✅ Created stratified sample: 12,498 complaints
💾 Saved sample to: ../data/sampled/complaints_sample.csv

📝 PART 2: TEXT CHUNKING
--------------------------------------------------
Chunking parameters: size=500, overlap=50 (matching pre-built)
Processing 12,498 complaints...
  Processed 1,000 complaints...
  Processed 2,000 complaints...
  Processed 3,000 complaints...
  Processed 4,000 complaints...
  Processed 5,00